# 1. Transforming data from Wikipedia page to dataframe

In [32]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [33]:
#Using pandas to read Wikipedia table into a dataframe
post_codes=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

Next the post codes with unassigned boroughs were removed and unassigned neighborhoods were renamed with their borough name

In [34]:
post_codes.columns = ['Postal Code','Borough','Neighborhoods']
post_codes=post_codes[post_codes.Borough!='Not assigned'].reset_index(drop=True)
for hoods in post_codes.Neighborhoods:
    if 'Neighborhoods'=='Not assigned':
        post_codes['Neighborhoods']=post_codes['Borough']
post_codes.head(12)

,Postal Code,Borough,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Finding the coordinates of Toronto, ON

In [4]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


# 2. Adding latitude and longitude coordinates of each neighborhood

Finding and storing coordinates for each postal code

In [5]:
import geocoder
Lat_list=[]
Lng_list=[]
for i in range(post_codes.shape[0]):
    address='{}, Toronto, Ontario'.format(post_codes.at[i,'Postal Code'])
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lng_list.append(g.latlng[1])

Instering latitude and longitude coordinates in the database

In [6]:
toronto_merged = post_codes
toronto_merged['Latitude']=Lat_list
toronto_merged['Longitude']=Lng_list
toronto_merged.head(12)

,Postal Code,Borough,Neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75188,-79.33036
1,M4A,North York,Victoria Village,43.73042,-79.31282
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66277,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
7,M3B,North York,Don Mills,43.74929,-79.36169
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70794,-79.31160
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(toronto_merged['Borough'].unique()),toronto_merged.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


In [8]:
#Remove Postal Code column because it isn't needed
toronto_merged=toronto_merged.drop(['Postal Code'], axis=1)
#Removed boroughs not containing "Toronto" to simplify map
toronto_data = toronto_merged[toronto_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Borough,Neighborhoods,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302
2,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818
3,Downtown Toronto,St. James Town,43.65143,-79.37557
4,East Toronto,The Beaches,43.67703,-79.29542


In [9]:
# creating map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for lat, lng, borough, neighborhoods in zip(post_codes['Latitude'], post_codes['Longitude'],post_codes['Borough'], post_codes['Neighborhoods']):
    label = '{}, {}'.format(neighborhoods, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# 3. Explore and cluster the neighborhoods in Toronto

In [10]:
# The code was removed by Watson Studio for sharing.

In [11]:
import requests
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Lat_list, 
    Lng_list, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=H3OAGZJZL2TKMBPT1HVWCBYAOY01URWO03CLNOVBG1PKUNJQ&client_secret=XHM3OC3ABFNVUT0OX2K55S4WPPYS5AY4CAJONMHMUCUZVRA4&v=20200605&ll=[43.75188000000003, 43.73042000000004, 43.655140000000074, 43.72321000000005, 43.66449000000006, 43.66277000000008, 43.81153000000006, 43.74929000000003, 43.707940000000065, 43.65736000000004, 43.70799000000005, 43.65279000000004, 43.78564000000006, 43.72184000000004, 43.68970000000007, 43.65143000000006, 43.69211000000007, 43.648900000000026, 43.765750000000025, 43.67703000000006, 43.64531000000005, 43.68784000000005, 43.768200000000036, 43.70909000000006, 43.65609000000006, 43.66878000000003, 43.769690000000026, 43.802850000000035, 43.74809000000005, 43.70127000000008, 43.64970000000005, 43.66509000000008, 43.74309000000005, 43.780970000000025, 43.76467000000008, 43.68811000000005, 43.64285000000007, 43.64873000000006, 43.72861000000006, 43.78102000000007, 43.73903000000007, 43.68375000000003, 43.6471000

Function to find nearby venues in each neighborhood

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhoods', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
# calling above function with toronto venue information
toronto_venues=getNearbyVenues(names=toronto_data.Neighborhoods,latitudes=toronto_data.Latitude,longitudes=toronto_data.Longitude)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [15]:
toronto_venues.head()

,Neighborhoods,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65514,-79.36265,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65514,-79.36265,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65514,-79.36265,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65514,-79.36265,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65514,-79.36265,Body Blitz Spa East,43.654735,-79.359874,Spa


In [16]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 222 unique categories.


Use onehot encoding to convert venue categories to numerical values

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhoods column to dataframe
toronto_onehot['Neighborhoods'] = toronto_venues['Neighborhoods'] 

# move neighborhoods column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhoods,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [18]:
#Take the mean of the frequency of each venue in their neighborhoods and grouping rows by neighborhood
toronto_grouped=toronto_onehot.groupby('Neighborhoods').mean().reset_index()
toronto_grouped

,Neighborhoods,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.00,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.015152,0.030303,0.000000,0.000000,0.015152,0.030303,0.000000,0.000000,0.0,0.015152,0.000000,0.000000,0.00,0.000000,0.030303,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.045455,0.090909,0.000000,0.000000,0.00,0.000000,0.00,0.015152,0.000000,0.015152,0.000000,0.000000,0.00,0.015152,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.015152,0.00,0.000000,0.000000,0.000000,0.00,0.015152,0.015152,0.015152,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.015152,0

In [19]:
toronto_grouped.shape

(38, 223)

In [20]:
# Top five venues for each neighborhood
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2    Restaurant  0.05
3          Café  0.03
4        Bakery  0.03


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.07
1  Coffee Shop  0.06
2          Bar  0.06
3   Restaurant  0.05
4    Gift Shop  0.03


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
          venue  freq
0   Coffee Shop  0.08
1         Hotel  0.05
2          Café  0.04
3    Restaurant  0.03
4  Concert Hall  0.02


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.07
2                Café  0.04
3                 Bar  0.04
4           Speakeasy  0.03


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.11
1   Clothing Store  0.08
2   Sandwich Place  0.04
3      Pizza Place  0.03
4

In [21]:
# Function to reverse the order of the venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating a new dataframe to store the top 10 venues for each neighborhood

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhood_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Farmers Market,Hotel,Beer Bar,Seafood Restaurant,Café,Cheese Shop,Bakery
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Sandwich Place,Gift Shop,Yoga Studio,Performing Arts Venue,Nightclub,Lounge
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Café,Restaurant,Tea Room,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Bookstore,Gym
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Café,Bar,Park,Restaurant,Pizza Place,Speakeasy,Bakery,Electronics Store
4,Central Bay Street,Coffee Shop,Clothing Store,Sandwich Place,Pizza Place,Bubble Tea Shop,Middle Eastern Restaurant,Electronics Store,Cosmetics Shop,Plaza,Park


## K-means Clustering

First set cluster numbers then run k-means clustering on Toronto neighborhoods

In [23]:
from sklearn.cluster import KMeans

kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhoods',1)

kmeans = KMeans(n_clusters=kclusters, random_state=0, n_init=12).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Creating a new dataframe that includes clusters and the top 10 venues for each neighborhood

In [24]:
# add clustering labels
neighborhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhood_venues_sorted.set_index('Neighborhood'), on='Neighborhoods')
toronto_merged = toronto_merged.dropna()
toronto_merged.head()

,Borough,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265,0.0,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Event Space,Spa,Electronics Store,Food Truck,Restaurant,Distribution Center
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302,0.0,Coffee Shop,Café,Sandwich Place,Park,College Theater,Burger Joint,Chinese Restaurant,Salon / Barbershop,Restaurant,Clothing Store
2,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818,0.0,Coffee Shop,Clothing Store,Hotel,Café,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Diner,Bookstore,Bubble Tea Shop
3,Downtown Toronto,St. James Town,43.65143,-79.37557,0.0,Coffee Shop,Café,Seafood Restaurant,Restaurant,American Restaurant,Cosmetics Shop,Cocktail Bar,Japanese Restaurant,Bakery,Moroccan Restaurant
4,East Toronto,The Beaches,43.67703,-79.29542,2.0,Health Food Store,Neighborhood,Pub,Trail,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


Creating a map using folium to visualize k-means clustering

In [25]:
import math

# creating map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# cluster color scheme
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# map markers
markers_colors = []

for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhoods'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],radius=5,popup=label,color=rainbow[int(cluster)-1],fill=True,fill_color=rainbow[int(cluster)-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters

### Cluster 0: Great spots for a coffee and a snack!

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Event Space,Spa,Electronics Store,Food Truck,Restaurant,Distribution Center
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Café,Sandwich Place,Park,College Theater,Burger Joint,Chinese Restaurant,Salon / Barbershop,Restaurant,Clothing Store
2,"Garden District, Ryerson",Coffee Shop,Clothing Store,Hotel,Café,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Diner,Bookstore,Bubble Tea Shop
3,St. James Town,Coffee Shop,Café,Seafood Restaurant,Restaurant,American Restaurant,Cosmetics Shop,Cocktail Bar,Japanese Restaurant,Bakery,Moroccan Restaurant
5,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Farmers Market,Hotel,Beer Bar,Seafood Restaurant,Café,Cheese Shop,Bakery
6,Central Bay Street,Coffee Shop,Clothing Store,Sandwich Place,Pizza Place,Bubble Tea Shop,Middle Eastern Restaurant,Electronics Store,Cosmetics Shop,Plaza,Park
7,Christie,Café,Grocery Store,Baby Store,Coffee Shop,Playground,Candy Store,Italian Restaurant,Athletics & Sports,Yoga Studio,Ethiopian Restaurant
8,"Richmond, Adelaide, King",Café,Hotel,Restaurant,Coffee Shop,Gym,Salad Place,Steakhouse,Asian Restaurant,American Restaurant,Japanese Restaurant
9,"Dufferin, Dovercourt Village",Park,Bus Line,Bar,Bank,Bakery,Pizza Place,Furniture / Home Store,Athletics & Sports,Middle Eastern Restaurant,Pet Store
10,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Hotel,Japanese Restaurant,Deli / Bodega,Park,Plaza,Boat or Ferry,Sandwich Place,Beer Bar,Sports Bar


### Cluster 1: Come eat a sandwich in the park!

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,"High Park, The Junction South",Sandwich Place,Park,Dog Run,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


### Cluster 2: Chill at the beach!

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,The Beaches,Health Food Store,Neighborhood,Pub,Trail,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


### Cluster 3: Yoga at the dog park!

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Park,Yoga Studio,Dog Run,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
21,"Forest Hill North & West, Forest Hill Road Park",Park,Yoga Studio,Dog Run,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


### Cluster 4: Come play/workout!

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,"North Toronto West, Lawrence Park",Garden,Playground,Gym Pool,Park,Yoga Studio,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
29,"Moore Park, Summerhill East",Playground,Gym,Park,Tennis Court,Yoga Studio,Distribution Center,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
33,Rosedale,Tennis Court,Playground,Park,Shop & Service,Bike Trail,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
